## Ejercicio 2.2: Modelo de regresión logística que predice si un individuo tendrá una cita en el día de San Valentín o no.

**Enunciado**

Se tiene el siguiente conjunto de datos sintéticos creado para permitir a los aspirantes a científicos o analistas de datos llevar a cabo expreimentos predictivos mediante la construcción de un modelo de aprendizaje automático que puede predecir si un individuo obtendrá una cita de San Valentín o no.

El conjunto de datos tiene las siguientes características:

- Name: el nombre ficticio de una persona.
- Age: la edad ficticia de una persona.
- Gender: el género de  una persona.
- Income: lo que la persona gana anualmente como ingreso.
- Appearance Score: esta característica representa una puntuación numérica que indica cómo se percibe a los individuos en función de su apariencia física.
- Interest Score: esta característica representa una puntuación numérica que indica qué tan alineados o compatibles están los intereses de una persona con los de los demás.
- Confidence Score: esta característica representa una puntuación numérica que indica el nivel de confianza percibido de un individuo.
- Educational Status: esta característica representa el logro educativo o el nivel de educación de un individuo.
- Job Type: esta característica representa el tipo de trabajo o situación laboral de un individuo.
- Valentine Date: esta es una característica binaria utilizada para la clasificación binaria (0, 1), que indica si un individuo tiene una fecha de San Valentín (1) o no (0).

El conjunto de datos se encuentra en el archivo: ***ValentineDataset.csv***.

Utilizando un conjunto de datos proporcionado, realizar lo siguiente:

<ol type="a">
<li>Cargar y explorar los datos.</li>
<li>Hallar la matriz de correlación de los datos para determinar las variables más significativas a usar en el modelo.</li>
<li>Dividir los datos en 90% para entrenamiento y el resto para prueba.</li>
<li>Implementar un modelo de regresión logística utilizando Python y entrenarlo.</li>
<li>Graficar la función de pérdida obtenida durante entrenamiento y analizarla.</li>
<li>Evaluar el modelo utilizando la matriz de confusión y las métricas: precision, accuracy y recall.</li>
</ol>



### Importar librerías necesarias

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

### Cargar y explorar los datos

In [ ]:
df = pd.read_csv('ValentineDataset.csv')
df = df.dropna()
df.head(20)

### Hallar la matriz de correlación de los datos para determinar las variables más significativas a usar en el modelo

In [ ]:
# Se eliminan columnas innecesarias o que no aportan información relevante
df = df.drop(columns=['Name'])

# One-hot encoding para variables categóricas
cat_cols = ['Gender', 'Educational_Status', 'Job_Type']
df_enc = pd.get_dummies(df, columns=cat_cols, drop_first=True)

# Se obtiene la matriz de correlación
corr_matrix = df_enc.corr()

# Se ordenan las variables por su correlación con la variable objetivo
corr_target = corr_matrix['Valentine_Date'].sort_values(ascending=False)
print("Correlación de cada variable con Valentine Date:\n", corr_target)

### División de datos

In [ ]:
X = df_enc.drop(columns=['Valentine_Date'])
y = df_enc['Valentine_Date']
X = X.astype(np.float64)
X = np.c_[np.ones(X.shape[0]), X]
# Normalización de los datos
X = (X - X.mean()) / X.std()
# Se dividen los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Funciones para el modelo de regresión

In [ ]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))


def loss(h, y):
    return -np.mean(y * np.log(h) + (1 - y) * np.log(1 - h))


def gradient_descent(X, y, alpha = 0.05, num_iterations = 1000):
    m = X.shape[0]
    weights = np.random.rand(X.shape[1])
    loss_values = []
    for i in range(num_iterations):
        z = np.dot(X, weights)
        sigmoid_values = sigmoid(z)
        gradient = np.dot(X.T, (sigmoid_values - y)) / m
        weights = weights - alpha * gradient
        current_loss = loss(sigmoid_values, y)
        loss_values.append(current_loss)
        if i % 100 == 0:
            print("Interacción:", i, "Pérdida:", current_loss)
    return weights, loss_values


def predict(X, weights):
    z = np.dot(X, weights)
    return np.where(sigmoid(z) >= 0.5, 1, 0)

### Entrenamiento del modelo

In [ ]:
weights, loss_values = gradient_descent(X_train, y_train, alpha=0.005, num_iterations=100000)
y_pred = predict(X_test, weights)

### Gráfico de la función de pérdida

In [ ]:
plt.hist(loss_values, bins=50)
plt.xlabel("Loss")
plt.ylabel("Frequency")
plt.title("Distribution of Loss")
plt.show()